In [6]:
from utils import load_data, check_gpu, check_dir
from clean_session import preprocess_sessions
import pandas as pd
import numpy as np
from ast import literal_eval

from tqdm import tqdm
import gc
from functools import partial
import matplotlib.pyplot as plt
from clean_session import preprocess_sessions
from create_model_inputs import prepare_data, compute_session_fts, create_model_inputs
import datetime
%matplotlib inline
%load_ext autoreload
%autoreload 2
def fprint(df, name):
    print(f'{name} shape: ({df.shape[0]:,}, {df.shape[1]})')
import pprint
pp = pprint.PrettyPrinter(indent=1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
nrows = 1000000
train = prepare_data('train', convert_action_type=False, nrows=nrows, recompute=False)
df = compute_session_fts(train, 'train')

[05-23 18:48:27 - utils-107 - load_data - INFO] Loading train using 1,000,000 rows (999,995 trimmed) which is 6.28% out of total train data
[05-23 18:48:27 - create_model_inputs-17 - flogger - INFO] raw train shape: (999,995, 12)
[05-23 18:48:27 - clean_session-56 - preprocess_sessions - INFO] Load from existing file: ./gbm_cache/preprocessed_train.snappy
[05-23 18:48:32 - create_model_inputs-66 - prepare_data - INFO] Sort df by user_id, session_id, timestamp, step
[05-23 18:48:39 - create_model_inputs-17 - flogger - INFO] Prepared train data shape: (5,200,153, 9)


KeyboardInterrupt: 

In [ ]:
df = df.groupby('session_id').last().reset_index()
# df['session_id_size'] = np.log(df['session_id_size'])
# df['cfs'] = df['current_filters_last_filters'].str.lower().str.split('|')

# df['prices'] = df['prices'].str.split('|')
# df['prices'] = df['prices'].apply(lambda x: [int(p) for p in x])

# df['time_steps'] = df['prices'].str.len()
# padding_mask = df['time_steps'] < 25
# df.loc[padding_mask, 'prices'] = df.loc[padding_mask, 'prices'].apply(lambda x: np.pad(x, (0, 25-len(x)),
#                                                                                        mode='constant'))
df['imps'] = df['impressions'].str.split('|')
df['imps'] = df['imps'].apply(lambda x: [int(i) for i in x])
# df.loc[padding_mask, 'impressions'] = (df.loc[padding_mask, 'impressions']
#                                          .apply(lambda x: np.pad(x, (0, 25-len(x)),mode='constant')))
df['reference'] = df['reference'].astype(int)
# filter out nan rows with reference_id not in impressions list, since if the true target in test
# is not in the impression list then it would not get evaluated
def assign_target(row):
    ref = row['reference']
    imp = list(row['imps'])
    if ref in imp:
        return imp.index(ref)
    else:
        return np.nan

df['target'] = df.apply(assign_target, axis=1)
# drop the ones whose reference is not in the impression list
df = df[df['target'].notna()].reset_index(drop=True)
df['target'] = df['target'].astype(int)
# def assign_last_ref_id(row, divide=True):
#     ref = row['reference_last_reference_id']
#     # although reference_id got converted to int, but the reference_last_reference_id was calculated
#     # when it was still str value, so here we look up the index in str of impressions
#     imp = [str(i) for i in row['imps']]

#     if pd.isna(ref):
#         return np.nan
#     else:
#         if ref in imp:
#             if divide:
#                 return (imp.index(ref) + 1) / len(imp)
#             else:
#                 return imp.index(ref) + 1
#         else:
#             return np.nan
# assign_last_ref_id_func = partial(assign_last_ref_id, divide=False)
# df['last_ref_ind'] = df.apply(assign_last_ref_id_func, axis=1)

### comapre session with size 1 and the rest

In [ ]:
df.head(2)

In [ ]:
ones = df[df['session_id_size']==1].reset_index(drop=True)
multiples = df[df['session_id_size']>1].reset_index(drop=True)

In [ ]:
oids = ones['session_id'].unique()
mids = multiples['session_id'].unique()

In [ ]:
df[df.session_id=='0000059a39020']

---
### ones

In [ ]:
display_cols = ['session_id', 'step', 'action_type', 'current_filters', 'reference', 'timestamp',
                'impressions', 'prices', 'target', 'session_id_size']
raw_display_cols = [c for c in train.columns if c in display_cols]

In [ ]:
pd.options.display.max_colwidth = 100

In [ ]:
one_rid = np.random.choice(oids, 1)[0]
print(one_rid)
df[df.session_id==one_rid][display_cols].reset_index(drop=True)

In [ ]:
# df[df.session_id==one_rid][display_cols].reset_index(drop=True).to_dict(orient='list')

In [ ]:
from colorama import Fore
dic = df[df.session_id==one_rid][display_cols].reset_index(drop=True).to_dict(orient='list')
prices = dic['prices'][0].split('|')
ref = str(dic['reference'][0])
imps = dic['impressions'][0].split('|')
# prices[imps.index(ref)] = f'{Fore.RED} {prices[imps.index(ref)]}' #{test_id}
prices[imps.index(ref)] = f'==={prices[imps.index(ref)]}===' #{test_id}
dic['prices'] = '|'.join(prices)
pp.pprint(dic)
# print(dic['prices'])

In [ ]:
print(f'{Fore.RED}{prices[imps.index(ref)]}')

In [ ]:
# # and the raw
# train[train.session_id==one_rid][raw_display_cols].reset_index(drop=True)

In [ ]:
# pp.pprint(train[train.session_id==one_rid][raw_display_cols].reset_index(drop=True).to_dict(orient='list'))

---
### Longer

In [ ]:
mid_rid = np.random.choice(mids, 1)[0]
print(mid_rid)
# df[df.session_id==mid_rid][display_cols].reset_index(drop=True)

In [ ]:
np.sort([int(i) for i in df[df.session_id==mid_rid]['prices'].str.split('|').values[0]])

In [ ]:
# pp.pprint(df[df.session_id==mid_rid][display_cols].reset_index(drop=True).to_dict(orient='list'))

In [ ]:
# the raw
train[train.session_id==mid_rid][raw_display_cols].reset_index(drop=True)

In [ ]:
pp.pprint(train[train.session_id==mid_rid][raw_display_cols].reset_index(drop=True).to_dict(orient='list'))

In [ ]:
# dup_mask = train[[c for c in train.columns if c not in ['timestamp', 'step']]].duplicated(keep=False)
# dup_mask.sum()

In [ ]:
# dup_cols = [c for c in train.columns if c not in ['timestamp', 'step']]
# # train[dup_cols].loc[train[dup_cols].shift(-1) != train[dup_cols]]
# train.groupby(dup_cols).apply(lambda x: )

In [ ]:
train.shape

In [ ]:
99508/train.shape[0]

### Look at test set
---

In [ ]:
test = load_data('tedt')

---
### Click-encodings

In [ ]:
from create_model_inputs import click_view_encoding
_ = click_view_encoding(sids=None, fold='testing', m=5)

In [ ]:
cv_enc = pd.read_csv('./cache/foldtesting_clickview_encodings.csv')

In [ ]:
cv_enc.shape

In [ ]:
cv_enc['clicked'].hist(bins=100)

---
### clickouts

In [ ]:
%%time
df = load_data('train', usecols=['session_id', 'action_type', 'impressions', 'reference'])

In [ ]:
df.shape

In [ ]:
df = df.drop_duplicates(subset=[c for c in df.columns if c != 'step']).reset_index(drop=True)

In [ ]:
df.shape

In [ ]:
%%time
# filter away sessions do not have clickout
def filter_away(df):
    return (df.action_type=='clickout item').sum() > 0
fids = df.groupby('session_id').apply(filter_away)

In [ ]:
df_has = df[df.session_id.isin(fids[fids].index)]

In [ ]:
df_has.shape

In [ ]:
# number of clickout rows
(df_has.action_type == 'clickout item').sum()

In [ ]:
last_df_has = df_has.groupby('session_id').last().reset_index()

In [ ]:
last_df_has.shape

In [ ]:
# average number of clickout per session
ctns = df_has[df_has.action_type=='clickout item'].groupby('session_id')['session_id'].size()

In [ ]:
ctns.head()

In [ ]:
s = df_has.groupby('session_id')['session_id'].size()

In [ ]:
s.head()

In [ ]:
# avg_clickout = ctns/s

In [ ]:
# avg_clickout.hist(bins=100)

In [ ]:
np.log(ctns).hist(bins=100)

In [ ]:
ctn

In [62]:
from datetime import datetime as dt
current_time = dt.now().strftime('%m-%d-%M')

In [63]:
current_time

'05-29-27'

In [2]:
sub = pd.read_csv('./subs/test_sub.csv')

In [46]:
sub = pd.read_csv('./subs/cat_sub_05-29.csv')

In [3]:
sub.head()

,session_id,impressions
0,0000059a39020,"[2251200, 924581, 4775012, 10090928, 2282660, ..."
1,000243bb22a50,"[49536, 149808, 2418418, 1122314, 149795, 5502..."
2,000356259e25a,"[948643, 34338, 18230, 100302, 1538159, 42402,..."
3,000356259e25a,"[948643, 34338, 18230, 100302, 1538159, 42402,..."
4,000356259e25a,"[948643, 34338, 18230, 100302, 1538159, 42402,..."


In [43]:
sub.shape

(486635, 3)

In [7]:
sub.loc[:, 'impressions'] = sub.loc[:, 'impressions'].apply(lambda x: literal_eval(x))


In [44]:
sub.iloc[0]

Unnamed: 0                                                     0
session_id                                         0000059a39020
impressions    [2251200, 924581, 4775012, 10090928, 2282660, ...
Name: 0, dtype: object

In [9]:
sub.iloc[0].impressions[0]

2251200

In [39]:
# from ast import literal_eval
# def convert2list(x):
# #     print('\n\n', x) 
# #     print(literal_eval(x), '\n\n')
#     try:
#         return literal_eval(x)
#     except:
#         return x.strip('([|])').split()
# test_sub.loc[:, 'impressions'] = test_sub.loc[:, 'impressions'].apply(convert2list)


In [49]:
s  = pd.read_csv('./data/submission_popular.csv')

In [5]:
s.shape

(253573, 5)

In [53]:
pd.options.display.max_colwidth=1000

In [54]:
s.head()

,user_id,session_id,timestamp,step,item_recommendations
0,000324D9BBUC,89643988fdbfb,1541593942,10,924795 106315 1033140 119494 101758 903037 1050792 97171 353141 2094950 923407 106307 1000915 97170 147907 1838901 1729121 2182130 904851 927351 4234718 2358690 2431104 1241375 152107
1,0004Q49X39PY,9de47d9a66494,1541641157,1,3505150 3812004 2227896 2292254 3184842 2227026 8153310 10213134 320746 2213014 1978919 4504242 1012049 595506 1953665 4486372 3508724 1984229 3202894 2714500 3202412 1987127 2714480 7769010 2712428
2,0004Q49X39PY,beea5c27030cb,1541561202,1,4476010 3505150 3812004 2227896 2292254 2227026 1736323 2926933 9343000 7101352 1953665 1984229 7984902 10024490 3202894 2508850 3843244 5052004 7794970 3766088 3823768 2712428 9017272 5497362 6030666
3,00071784XQ6B,9617600e1ba7c,1541630328,2,22854 3067559 22721 22713 16121 22772 22727 22764 150882 22716 150825 22714 22711 22798 45096 4082698 102569 5889794 102568 1478189 22819 22797 22705 22848 3498754
4,0008BO33KUQ0,2d0e2102ee0dc,1541636411,6,9857656 5849628 655716 1352530 502066 1405084 2176280 507861 2143854 4342348 1241177 1669587 6451832 977727 6147882 1008737 4272108 3133206 1887759 1830637 3133204 4984730 1888653 1944129 1496381


In [55]:
sub.head()

,user_id,session_id,timestamp,step,item_recommendations
0,P2L70S1Y60IF,0000059a39020,1541719950,1,2251200 924581 895299 10090928 4775012 2282660 1042070 904419 4393408 716761 9327438 494271 1725409 1574393 9814964 449646 494591 1255814 1725407 1532985 106496 1552635 1552439 394496 103453
1,SKDEGHT3I151,0000b9394297b,1541606244,2,1044228 5670638 5078576 6033244 6739860 11245146 7917004 5875672 2253886 5483610 7731204 10075218 6040960 6706660 9803684 8111266 8314306 0 7672042 0 0 0 0 0 0
2,V3SQM6X57BNX,0001650138d30,1541614783,1,44394 44247 44103 44155 44404 43972 44292 1322086 44288 843686 974755 44496 44421 44391 1253177 44904 43995 138567 43764 1285738 914037 101050 44289 44101 43959
3,13UA01X5MOHZ,000177b850519,1541574580,4,17011 17266 16122 17403 17295 17535 17040 17578 153593 1218858 4897876 16831 2232136 2612598 1977627 16842 3188920 17579 85123 17028 2881132 5050146 9117602 6001568 3140558
4,K0LALBSISIAC,00017b3b2c136,1541565393,11,6698142 6298194 7952264 5717238 2859570 2811944 3984784 5723818 7240314 4341718 9187692 401776 2810449 4277618 8175520 9672594 509856 5486494 2565452 1101646 7198288 2224450 1122702 3492498 1946813


In [56]:
sub[sub.session_id=='89643988fdbfb']

,user_id,session_id,timestamp,step,item_recommendations
136070,000324D9BBUC,89643988fdbfb,1541593942,10,1241375 923407 119494 152107 147907 101758 1729121 1050792 2182130 4234718 97171 353141 927351 1000915 2094950 106307 1033140 106315 1838901 924795 2358690 904851 903037 2431104 97170


In [57]:
to_sub = sub.groupby('session_id').last().reset_index()

In [9]:
to_sub.shape

(253573, 5)

In [10]:
to_sub.shape

(253573, 5)

In [11]:
to_sub.head()

,session_id,user_id,timestamp,step,item_recommendations
0,0000059a39020,P2L70S1Y60IF,1541719950,1,"[2251200, 924581, 4775012, 10090928, 2282660, ..."
1,0000b9394297b,SKDEGHT3I151,1541606244,2,[ 1044228 5670638 5875672 6033244 7917004 ...
2,0001650138d30,V3SQM6X57BNX,1541614783,1,"[44394, 44103, 44247, 44155, 43972, 44292, 444..."
3,000177b850519,13UA01X5MOHZ,1541574580,4,"[17011, 16122, 17403, 17040, 17266, 17295, 175..."
4,00017b3b2c136,K0LALBSISIAC,1541565393,11,"0 0 0 0 0 0 0 [5723818, 4341718, 6298194, 6698..."


In [12]:
s.head()

,user_id,session_id,timestamp,step,item_recommendations
0,000324D9BBUC,89643988fdbfb,1541593942,10,924795 106315 1033140 119494 101758 903037 105...
1,0004Q49X39PY,9de47d9a66494,1541641157,1,3505150 3812004 2227896 2292254 3184842 222702...
2,0004Q49X39PY,beea5c27030cb,1541561202,1,4476010 3505150 3812004 2227896 2292254 222702...
3,00071784XQ6B,9617600e1ba7c,1541630328,2,22854 3067559 22721 22713 16121 22772 22727 22...
4,0008BO33KUQ0,2d0e2102ee0dc,1541636411,6,9857656 5849628 655716 1352530 502066 1405084 ...


In [13]:
s[s.session_id=='0000059a39020']

,user_id,session_id,timestamp,step,item_recommendations
182944,P2L70S1Y60IF,0000059a39020,1541719950,1,1042070 106496 895299 394496 494271 716761 125...


In [15]:
to_sub.iloc[4].item_recommendations

'0 0 0 0 0 0 0 [5723818, 4341718, 6298194, 6698142, 7952264, 2859570, 5717238, 3984784, 2811944, 401776, 7240314, 509856, 9187692, 8175520, 9672594, 4277618, 5486494, 1101646, 2810449, 2224450, 1122702, 2565452, 3492498, 7198288, 1946813] 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0'

In [33]:
a = pd.DataFrame({'A': [[1,2,3], [0, 0, 0], [-1,-2,-3]], 
                  'B': [np.array([1,2,3]), [0, 0, 0], [-1,-2,-3]]})
a.to_csv('./dummy.csv')

In [34]:
pd.read_csv('./dummy.csv')

,Unnamed: 0,A,B
0,0,"[1, 2, 3]",[1 2 3]
1,1,"[0, 0, 0]","[0, 0, 0]"
2,2,"[-1, -2, -3]","[-1, -2, -3]"
